<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/Citibike_Folium_Pydeck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

The dataset is stored in the `public` database on the MySQL Server `db.ipeirotis.org`.

In [1]:
!pip3 install  -U -q  PyMySQL sqlalchemy pandas geopandas seaborn matplotlib pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 907.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import geopandas as gpd
import seaborn as sns
import matplotlib as plt
import pydeck as pdk
import folium

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Database Connection

In [5]:
conn_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user     = 'root',
    password = 'loPOMtFqwXeqENPGwpblNqSrNHclRtyL',
    host     = 'roundhouse.proxy.rlwy.net',
    port     = 34655,
    db       = 'railway'
)

engine = create_engine(conn_string)

In [6]:
conn_string_panos = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user = 'student'
    , password = 'dwdstudent2015'
    , host = 'db.ipeirotis.org'
    , port = 3306
    , db = 'public'
    , encoding = 'utf8'
)

engine_panos = create_engine(conn_string_panos)

# Helper Functions

In [7]:
# Used to fetch data from a table
def sql_to_pd(engine, sql):
    with engine.connect() as connection:
        results = pd.read_sql(
            text(sql)
            , con = connection)
    return results

# Citibike Data

## Trips by Bike Type

### 2023

In [8]:
sql = '''
  select
    rideable_type
    , date_format(started_at, '%Y-%m') as date
    , count(*) as num_rides
  from
    citibike_cjteam
  where
    started_at < '2024-01-01'
    and rideable_type != 'docked_bike'
  group by
    1, 2
'''

# Using panos' server here
df = sql_to_pd(engine_panos, sql)
df.head()

,rideable_type,date,num_rides
0,classic_bike,2023-04,2381013
1,electric_bike,2023-04,457579
2,classic_bike,2023-12,2195471
3,electric_bike,2023-12,78194
4,classic_bike,2023-11,2813771


In [9]:
# Update the code below to make the electric group be blue and classic be red

fig = px.bar(df
             , x = 'date'
             , y = 'num_rides'
             , title = '2023 Trips by Bike Type'
             , color = 'rideable_type'
             , color_discrete_map = {
                'classic_bike': px.colors.qualitative.Plotly[1],
                'electric_bike': px.colors.qualitative.Plotly[0]
              }
             , barmode = 'group').update_layout(
                 xaxis_title = 'Date'
                 , yaxis_title = '# Bike Trips'
                 , legend_title = 'Bike Type'
            )
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1992: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


### 2024

In [10]:
sql = '''
  select
    rideable_type
    , date_format(started_at, '%Y-%m') as date
    , count(*) as num_rides
  from
    citibike_cjteam
  group by
    1, 2
'''

# Using panos' server here
df = sql_to_pd(engine, sql)
df.head()

KeyboardInterrupt: 

In [ ]:
fig = px.bar(df
             , x = 'date'
             , y = 'num_rides'
             , title = '2024 Trips by Bike Type'
             , color = 'rideable_type'
             , color_discrete_map = {
                'classic_bike': px.colors.qualitative.Plotly[1],
                'electric_bike': px.colors.qualitative.Plotly[0]
              }
             , barmode = 'group').update_layout(
                 xaxis_title = 'Date'
                 , yaxis_title = '# Bike Trips'
                 , xaxis = dict(
                    dtick = 'M1'
                  )
                 , legend_title = 'Bike Type'
            )
fig.show()

### Pie Chart of Usage by Bike Type

In [ ]:
sql = '''
select
    rideable_type
    , count(*) as num_rides
from
    citibike_cjteam
group by
    rideable_type
'''

df = sql_to_pd(engine, sql)
df.head()

In [ ]:
# Create a pie chart
fig = px.pie(df
             , values = 'num_rides'
             , names = 'rideable_type'
             , title = '% of Rides by Bike Type'
)

# Show the pie chart
fig.show()

## Trip Routes

### Starting Station

In [11]:
sql = '''
select
    start_station_name
    , count(*) as num_rides
from
    citibike_cjteam
where
    start_station_name is not null
group by
    start_station_name
order by
  num_rides desc
limit 10
'''

df = sql_to_pd(engine, sql)
df.head()

,start_station_name,num_rides
0,W 21 St & 6 Ave,58437
1,8 Ave & W 31 St,47904
2,University Pl & E 14 St,45646
3,Broadway & W 58 St,45238
4,Broadway & W 25 St,44851


In [12]:
fig = px.bar(df
             , x = 'num_rides'
             , y = 'start_station_name'
             , title = 'Top 10 Most Popular Starting Station'
             , orientation = 'h').update_layout(
                 xaxis_title = '# Bike Rides'
                 , yaxis_title = 'Starting Station Name'
            ).update_yaxes(categoryorder = 'total ascending')
fig.show()

### Ending Station

In [ ]:
sql = '''
select
    end_station_name
    , count(*) as num_rides
from
    citibike_cjteam
where
    end_station_name is not null
group by
    1
order by
  num_rides desc
limit 10
'''

df = sql_to_pd(engine, sql)
df.head()

In [ ]:
fig = px.bar(df
             , x = 'num_rides'
             , y = 'end_station_name'
             , title = 'Top 10 Most Popular Ending Station'
             , orientation = 'h').update_layout(
                 xaxis_title = '# Bike Rides'
                 , yaxis_title = 'Ending Station Name'
            ).update_yaxes(categoryorder = 'total ascending')
fig.show()

### Start <> Ending Route

In [ ]:
sql = '''
select
    start_station_name
    , end_station_name
    , count(*) as num_rides
from
    citibike_cjteam
where
    1 = 1
    and start_station_name is not null
    and end_station_name is not null
group by
    1, 2
order by
  num_rides desc
limit 10;
'''

df = sql_to_pd(engine, sql)
df.head()

## Trip Duration

### Duration by Bike Type

In [ ]:
# Get the trip duration for each bike type
sql = '''
    select
        rideable_type
        , timestampdiff(minute, started_at, ended_at) as trip_duration
        , count(*) as num_rides
    from
        railway.citibike_cjteam
    group
        by 1, 2
    having
        trip_duration >= 0;
'''

# Use the sql_to_pd function to run the query and convert the result to a pandas dataframe
df = sql_to_pd(engine, sql)
df.head()

In [ ]:
def weighted_median(df):
    df_sorted = df.sort_values('trip_duration')
    cumsum = df_sorted['num_rides'].cumsum()
    total_sum = df_sorted['num_rides'].sum()
    midpoint = total_sum / 2.0
    return df_sorted.loc[cumsum >= midpoint, 'trip_duration'].iloc[0]

# Calculate weighted average trip duration
weighted_avg_trip_duration = df.groupby('rideable_type').apply(lambda x: (x['trip_duration'] * x['num_rides']).sum() / x['num_rides'].sum(), include_groups = False)
print("Weighted Average Trip Duration per Bike Type:")
print(weighted_avg_trip_duration)
print('\n')

# Calculate weighted median trip duration per rideable_type
weighted_median_trip_duration = df.groupby('rideable_type').apply(weighted_median, include_groups = False)
print("Weighted Median Trip Duration per Bike Type:")
print(weighted_median_trip_duration)

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'trip_duration'
                   , y = 'num_rides'
                   , color = 'rideable_type'
                   , title = 'Distribution of Trip Durations'
                   , labels = {'trip_duration': 'Trip Duration (minutes)', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 50]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.8, xbins = dict(size = 1))

# Show the plot
fig.show()

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'trip_duration'
                   , y = 'num_rides'
                   , histnorm = 'probability density'
                   , color = 'rideable_type'
                   , title = 'Probability Distribution of Trip Durations'
                   , labels = {'trip_duration': 'Trip Duration (minutes)', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 50]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.8, xbins = dict(size = 1))

# Show the plot
fig.show()

### Duration by Member Type

In [ ]:
# Get the trip duration by member type
sql = '''
    select
        member_casual
        , timestampdiff(minute, started_at, ended_at) as trip_duration
        , count(*) as num_rides
    from
        railway.citibike_cjteam
    group
        by 1, 2
    having
        trip_duration >= 0;
'''

# Use the sql_to_pd function to run the query and convert the result to a pandas dataframe
df = sql_to_pd(engine, sql)
df.head()

In [ ]:
# Calculate weighted average trip duration
weighted_avg_trip_duration = df.groupby('member_casual').apply(lambda x: (x['trip_duration'] * x['num_rides']).sum() / x['num_rides'].sum(), include_groups = False)
print("Weighted Average Trip Duration per Member Type:")
print(weighted_avg_trip_duration)
print('\n')

# Calculate weighted median trip duration per member type
weighted_median_trip_duration = df.groupby('member_casual').apply(weighted_median, include_groups = False)
print("Weighted Median Trip Duration per Member Type:")
print(weighted_median_trip_duration)

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'trip_duration'
                   , y = 'num_rides'
                   , color = 'member_casual'
                   , title = 'Distribution of Trip Durations by Member Type'
                   , labels = {'trip_duration': 'Trip Duration (minutes)', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 50]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.8, xbins = dict(size = 1))

# Show the plot
fig.show()

In [ ]:
fig = px.box(df
             , x = 'member_casual'
             , y = 'trip_duration'
             , color = 'member_casual'
             , title = 'Distribution of Trip Durations by Member Type'
             , labels = {'trip_duration': 'Trip Duration (minutes)', 'member_casual': 'Member Type'}
             ).update_layout(showlegend = False)

# Show the plot
fig.show()

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'trip_duration'
                   , y = 'num_rides'
                   , color = 'member_casual'
                   , title = 'Distribution of Trip Durations by Member Type'
                   , labels = {'trip_duration': 'Trip Duration (minutes)', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 50]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.8, xbins = dict(size = 1))

# Show the plot
fig.show()

## Trips by Day of Week

In [ ]:
sql = '''
    select
        case
            when dayofweek(started_at) = 1 or dayofweek(started_at) = 1 then 'Weekend'
            else 'Weekday'
        end as wknd_wkdy
        , hour(started_at) as hour_of_day
        , count(*) as num_rides
    from
        citibike_cjteam
    where
        1 = 1
    group by
        1, 2;
'''

df = sql_to_pd(engine, sql)
df.head()

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'hour_of_day'
                   , y = 'num_rides'
                   , color = 'wknd_wkdy'
                   , title = 'Trip Count by Day of Week'
                   , labels = {'hour_of_day': 'Hour of Day', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 23]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.65, xbins = dict(size = 1))

# Show the plot
fig.show()

In [ ]:
# Create the histogram
fig = px.histogram(df
                   , x = 'hour_of_day'
                   , y = 'num_rides'
                   , histnorm = 'probability density'
                   , color = 'wknd_wkdy'
                   , title = 'Probability Distribution of Trip Durations'
                   , labels = {'hour_of_day': 'Hour of Day', 'num_rides': 'Number of Rides'}
                   , range_x = [0, 24]
                   ).update_layout(barmode = 'overlay'
                   ).update_traces(opacity = 0.65, xbins = dict(size = 1))

# Show the plot
fig.show()

## Visualizing Trips by Hour of Day

In [24]:
# 2024 Citibike Data aggregated to starting lat and lon

sql = '''
WITH citi_bike AS (
  SELECT start_station_id,
    DATE(started_at) AS date,
    DAYOFWEEK(started_at) AS day_of_week,
    HOUR(started_at) AS hour,
    COUNT(*) AS total_rides,
    AVG(start_lat) AS latitude,
    AVG(start_lng) AS longitude
  FROM railway.citibike_cjteam
  WHERE started_at >= '2024-05-01'
  GROUP BY start_station_id, DAYOFWEEK(started_at), HOUR(started_at), DATE(started_at)
)
SELECT
  day_of_week,
  hour,
  start_station_id,
  latitude,
  longitude,
  AVG(total_rides) AS avg_rides
FROM citi_bike
GROUP BY day_of_week, hour, start_station_id, latitude, longitude;
'''

df_citibik_juli = sql_to_pd(engine, sql)

df_citibik_juli['hour'] = 24 * (df_citibik_juli['day_of_week']-1) + df_citibik_juli['hour']

In [25]:
df_citibik_juli

,day_of_week,hour,start_station_id,latitude,longitude,avg_rides
0,1,0,None,40.725000,-74.005001,2.0
1,1,0,None,40.734999,-73.985001,2.0
2,1,0,None,40.743333,-73.993334,3.0
3,1,0,None,40.759998,-74.000000,1.0
4,1,0,2782.02,40.635860,-74.019836,1.0
...,...,...,...,...,...,...
847356,7,167,8879.02,40.886154,-73.899826,1.0
847357,7,167,8879.02,40.886322,-73.899719,1.0
847358,7,167,8879.03,40.885273,-73.900627,2.0
847359,7,167,8879.03,40.885324,-73.900566,3.0


In [26]:
df_citibik_juli2 = df_citibik_juli[['latitude', 'longitude', 'hour', 'avg_rides']]

In [28]:
# Folium map of rides
from folium.plugins import HeatMapWithTime
from collections import defaultdict, OrderedDict

# Create a map centered at the average latitude and longitude of the data
m = folium.Map(location=[df_citibik_juli['latitude'].mean(), df_citibik_juli['longitude'].mean()], zoom_start=12)

# Aggregate data based on latitude, longitude, and hour
# Filter to just the columns latitude, longitude, hour, and avg_rides
# df_citibik_juli2 = df_citibik_juli[['latitude', 'longitude', 'hour', 'avg_rides']]
# df_citibik_juli2 = df_citibik_juli2.groupby(['latitude', 'longitude', 'hour']).mean().reset_index()

data = defaultdict(list)
for r in df_citibik_juli2.itertuples():
    data[r.hour].append([r.latitude, r.longitude, r.avg_rides/50])

data = OrderedDict(sorted(data.items(), key=lambda t: t[0]))

hm = HeatMapWithTime(data=list(data.values()),
                     index=list(data.keys()),
                     radius=10,
                     auto_play=True)


hm.add_to(m)
m
m.save("heatmap.html")

## Weather Data

WT03 - Thunder

WESF - Water equivalent of snowfall

WT04 - Ice pellets, sleet, snow pellets, or small hail"

PRCP - Precipitation

WT05 - Hail (may include small hail)

WT06 - Glaze or rime

WT08 - Smoke or haze

SNWD - Snow depth

WDF2 - Direction of fastest 2-minute wind

WDF5 - Direction of fastest 5-second wind

PGTM - Peak gust time

WT11 - High or damaging winds

TMAX - Maximum temperature

DAPR - Number of days included in the multiday precipitation total (MDPR)

WSF2 - Fastest 2-minute wind speed

WSF5 - Fastest 5-second wind speed

SNOW - Snowfall

TOBS - Temperature at the time of observation

AWND - Average wind speed

WT01 - Fog, ice fog, or freezing fog (may include heavy fog)

WESD - Water equivalent of snow on the ground

WT02 - Heavy fog or heaving freezing fog (not always distinguished from fog)

TAVG - Average Temperature.

TMIN - Minimum temperature

MDPR - Multiday precipitation total (use with DAPR and DWPR, if available)

### Weather Stations in NYC

In [ ]:
# Weather data - longitude, latitude, and average precipitation
sql = '''
select
    distinct station
    , name
    , latitude
    , longitude
from
    nyc_weather_cjteam
where
    name in ('WORLD TRADE CENTER, NY US', 'BROOKLYN 2.4 WSW, NY US', 'BROOKLYN 3.1 NW, NY US', 'NY CITY CENTRAL PARK, NY US');
'''

df = sql_to_pd(engine, sql)
df.head()

In [ ]:
df['coordinates'] = df[['longitude', 'latitude']].apply(lambda x: [x[0], x[1]], axis=1)

# Create a ScatterplotLayer
layer = pdk.Layer(
    'ScatterplotLayer'
    , df
    , pickable = True
    , get_position = 'coordinates'
    , get_radius=200
    , get_fill_color=[255, 0, 0]
    , get_line_color=[0, 0, 0]
    , line_width_min_pixels=1
)

view_state = pdk.ViewState(
    latitude = 40.7,
    longitude = -73.9880,
    zoom = 11.3,
    bearing = 0,
    pitch = 45
)

# Render
r = pdk.Deck(
    layers = [layer]
    , initial_view_state = view_state
    , tooltip = {'text': 'Station: {station}\nName: {name}\nCoordinates: {coordinates}'}
)

# Save the map to an HTML file
r.to_html('weather_stations.html')

### Total Weekly Rides vs Avg Weekly Precipitation

In [ ]:
# Weather data - longitude, latitude, and average precipitation
sql = '''
with trips as (
    select
        weekofyear(started_at) as day_date
        , count(*) as total_rides
    from
        citibike_cjteam
    group by
        1
), daily_weather as (
    select
        weekofyear(date) as day_date
        , avg(prcp) as avg_prcp
    from
        nyc_weather_cjteam
    where
        1 = 1
        and name in ('WORLD TRADE CENTER, NY US', 'BROOKLYN 2.4 WSW, NY US', 'BROOKLYN 3.1 NW, NY US', 'NY CITY CENTRAL PARK, NY US')
        and date >= '2024-01-01'
    group by
        1
)
select
    daily_weather.day_date
    , total_rides
    , avg_prcp
from
    trips
    left join daily_weather
        on trips.day_date = daily_weather.day_date
order by 1 asc;
'''

df = sql_to_pd(engine, sql)
df.head()

In [ ]:
# Order the df
df = df.sort_values(by = ['day_date'])

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x = df['day_date'], y = df['total_rides'], name = 'Total Rides'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = df['day_date'], y = df['avg_prcp'], name = 'Avg Precip'),
    secondary_y=True,
)

fig.update_layout(
    title = 'Weekly Total Rides vs. Avg Precipitation in 2024',
    xaxis = dict(
        title = 'Week of Year'
    ),
    yaxis = dict(
        title = 'Total Rides',
        tickfont=dict(
            color='blue'
        )
    ),
    yaxis2 = dict(
        title = 'Avg Precipitation (inches)',
        tickfont = dict(
            color = 'red'
        ),
        overlaying = 'y',
        side = 'right'
    )
)

### Impact of Weather on Proportion of Bike Rides

In [ ]:
# Query to get bike data
bike_query = '''
select
    count(*) as total_rides,
    rideable_type,
    year(started_at) as year,
    month(started_at) as month
from
  railway.citibike_cjteam
where
  rideable_type != 'docked_bike'
group by
  year, month, rideable_type
order by
  year, month asc;
'''

# Execute bike query
df_bikenumbers = sql_to_pd(engine, bike_query)

# Query to get weather data
weather_query = '''
select
  year(date) as year
  , month(date) as month
  , avg(prcp) as avg_prcp
from
  nyc_weather_cjteam
where
  1 = 1
  and name in ('WORLD TRADE CENTER, NY US', 'BROOKLYN 2.4 WSW, NY US', 'BROOKLYN 3.1 NW, NY US', 'NY CITY CENTRAL PARK, NY US')
group by
  year, month;
'''

# Execute weather query
df_weather = sql_to_pd(engine, weather_query)

# Join the dataframes on year and month
combined_df = pd.merge(df_bikenumbers, df_weather, on=['year', 'month'])

# First, calculate the total rides per precipitation level and bike type
grouped = combined_df.groupby(['avg_prcp', 'rideable_type']).agg({'total_rides': 'sum'}).reset_index()

# Next, calculate the total rides per precipitation level
total_rides_per_precip = grouped.groupby('avg_prcp')['total_rides'].transform('sum')

# Calculate the proportion of rides per bike type at each precipitation level
grouped['proportion'] = grouped['total_rides'] / total_rides_per_precip

# Check the DataFrame
print(grouped.head())

# Display the first few rows of the combined dataframe
combined_df.head()

In [ ]:
# Using plotly plot the avg_prcp vs the proportion from combined_df grouped by rideable_type
fig = px.line(grouped
              , x = 'avg_prcp'
              , y = 'proportion'
              , title = 'Proportion of Rides by Precipitation Level and Bike Type'
              , color = 'rideable_type').update_layout(
                 xaxis_title = 'Average Precipitation (inches)'
                 , yaxis_title = 'Proportion of Rides'
                 , legend_title = 'Bike Type'
            )

# Show the plot
fig.show()

## Collisions data

### Collisions Involving Cyclists

In [ ]:
sql = '''
select
    latitude
    , longitude
    , count(collision_id) as num_collisions
from
    collisions_cjteam
where
    year(crash_date) = 2024
    and latitude is not null
    and longitude is not null
    and (number_of_cyclist_injured > 0 or number_of_cyclist_killed > 0)
group by
    1, 2;
'''

df = sql_to_pd(engine, sql)
df['coordinates'] = df[['longitude', 'latitude']].apply(lambda x: [x[0], x[1]], axis=1)
df.head()

### Location of Collisions

- 3	decimal places is 111 m of accuracy
- 4	decimal places is 11.1 m of accuracy

In [ ]:
layer = pdk.Layer(
    'GridLayer',
    df,
    pickable = True,
    extruded = True,
    cell_size = 200,
    elevation_scale = 2,
    get_elevation = 'num_collisions',
    get_position = 'coordinates',
)

view_state = pdk.ViewState(
    latitude = 40.7
    , longitude = -73.9880
    , zoom = 11
    , bearing = 0
    , pitch = 45)

r = pdk.Deck(
    layers = [layer],
    initial_view_state = view_state,
    tooltip = {'text': '{position}\nCount: {count}'},
)
r.to_html('grid_layer.html')